# Parameter recovery

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
plt.rcParams.update({'font.size': 14})

# Initialize random number generator
np.random.seed(1234)

import sys
print("Python version:\n{}\n".format(sys.version))

Python version:
3.7.4 (default, Aug 13 2019, 15:17:50) 
[Clang 4.0.1 (tags/RELEASE_401/final)]



In [2]:
%load_ext autoreload
%autoreload 2

## Define `param_recovery_func`

`param_recovery_func` is the core function that we have to write and provide to the `ParamRecovery2d` class. In our case parameter recovery will consist of a few simple steps of simulating data and conducting parameter estimation.

This is our high level function...

In [4]:
def param_recovery_func(params_true):
    data = simulate_data(params_true)
    params_inferred = param_estimation(data, sample_options)
    row_data = pd.concat([params_true, params_inferred], axis=1)
    return row_data

We then need to define the functions `simulate_data` and `param_estimation`.

### Define how we simulate data

In [5]:
from designs import generate_designs
from models import ModifiedRachlin

generate_responses = ModifiedRachlin.generate_responses

def simulate_data(params_true, ϵ=0.01):
    designs = generate_designs()
    responses, _ = generate_responses(designs, params_true, ϵ)
    return pd.concat([designs, responses], axis=1)

/anaconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


### Define parameter estimation

Second, we define how we conduct parameter estimation. Note that this is a _non_ hierarchical model which does parameter estimation for a single participant/experiment.

In [6]:
from model_solo_modified_rachlin import parameter_estimation

sample_options={"tune": 1000,
                "draws": 2000,
                "chains": 2,
                "cores": 2,
                "nuts_kwargs": {"target_accept": 0.95}}

params_inferred = param_estimation(data)

In [7]:
# def parameter_estimation(data):
#     model = generate_model(data)
#     with model:
#         trace = pm.sample(**sample_options)
#     return extract_info_from_trace(trace)

## Run the parameter recovery
This is where the hard simulation work happens. This will take some time to compute.

In [8]:
from param_recovery import ParamRecovery2d

In [10]:
my_recovery = ParamRecovery2d(
    params={
        "logk": np.linspace(-5, 1, num=3),
        "logs": np.linspace(np.log(0.5), np.log(3), num=3),
    },
    param_recovery_func=param_recovery_func,
    n_simulations=3,
)

Commencing parameter recovery. Might take some time...


NameError: name 'PARAMS' is not defined

In [ ]:
my_recovery.true_params

In [ ]:
my_recovery.plot()